In [21]:
import pandas as pd
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
import random

### Read Data

In [22]:
df = pd.read_excel("data/wordFrequency.xlsx", sheet_name = "1 lemmas", index_col = "rank")
df = df[["lemma", "PoS", "freq"]]
df["lemma"] = df["lemma"].str.lower()

### Filter PoS

In [23]:
stop_words = set(stopwords.words('english'))
df = df[~df["lemma"].isin(stop_words)]
df = df[df["PoS"].isin(["n", "v", "j", "r"])]
df["PoS"] = df["PoS"].replace({"n": wn.NOUN, "v": wn.VERB, "j": wn.ADJ, "r": wn.ADV})

### Generate Synonym

In [24]:
def get_synonym(word, pos):
    try:
      synsets = wn.synsets(word, pos)
      random.shuffle(synsets)
      for synset in synsets:
            lemmas = synset.lemmas()
            random.shuffle(lemmas)
            for lemma in lemmas:
                  lemma = lemma.name()
                  if lemma.isalpha() and lemma != word:
                        return lemma, synset
    except:
          pass
    return None, None

In [25]:
df["synonym"] = None
df["synset"] = None
for idx, row in df.iterrows():
    df.loc[idx, "synonym"], df.loc[idx, "synset"] = get_synonym(df.loc[idx, "lemma"], df.loc[idx, "PoS"])

In [26]:
df = df.dropna(axis = 0, how = "any")

### Generate Random Pairs

In [27]:
random_word_dic = {wn.NOUN: list(wn.all_synsets("n")),
                   wn.VERB: list(wn.all_synsets("v")),
                   wn.ADJ: list(wn.all_synsets("a")),
                   wn.ADV: list(wn.all_synsets("r"))}

In [28]:
def get_random_word_by_pos(pos, target_word):
    synsets = random_word_dic[pos]
    while True:
            random_synset = random.choice(synsets)
            random_word = random.choice(random_synset.lemmas()).name()
            if random_word.isalpha() and random_word != target_word: 
                return random_word
    return None

In [29]:
df["random"] = None
for idx, row in df.iterrows():
    df.loc[idx, "random"]= get_random_word_by_pos(df.loc[idx, "PoS"], df.loc[idx, "lemma"])

In [30]:
df["lemma"] = df["lemma"].str.lower()
df["synonym"] = df["synonym"].str.lower()
df["random"] = df["random"].str.lower()
df.head()

,lemma,PoS,freq,synonym,synset,random
rank,,,,,,
26,say,v,4096416,state,Synset('state.v.01'),emplane
31,go,v,3546732,last,Synset('survive.v.01'),vellicate
34,get,v,3347615,contract,Synset('contract.v.04'),report
39,know,v,2761628,cognise,Synset('know.v.01'),saccharify
50,make,v,2290830,produce,Synset('produce.v.02'),trespass


In [31]:
df.to_csv("data/4000_word_pairs.csv")